In [1]:
import nltk
from nltk.corpus import stopwords

In [3]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


KeyboardInterrupt: 

In [4]:
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [5]:
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
ps = PorterStemmer()
test = "importantance of caving as explained by cavers"

words = word_tokenize(test)
for w in words:
    print(ps.stem(w))

important
of
cave
as
explain
by
caver


In [71]:
# hello:
# hello
# o$hell
# lo$hel
# llo$he
# ello$h

In [72]:
def produce_rotations(word):
    res = [word]
    term = "$" + word
    for i in range(len(word) - 1):
        term = term[-1] + term[:-1]
        res.append(term)
    return res

In [73]:
print(produce_rotations("hello"))

['hello', 'o$hell', 'lo$hel', 'llo$he', 'ello$h']


In [98]:
def rotate(wildcard):
    if wildcard[-1] == "*":
        return wildcard[:-1], True
    term = wildcard + "$"
    for i, l in enumerate(term, 1):
        if l == "*":
            return term[i:] + term[:i-1], True
    else:
        return wildcard, False

In [109]:
print(rotate("hello*"))

('hello', True)


In [103]:
words = ["hello", "bye", "brother", "hellow"]
db = {w: {"rotations": produce_rotations(w),
         "postings": produce_rotations(w)} for w in words}

In [104]:
db

{'hello': {'rotations': ['hello', 'o$hell', 'lo$hel', 'llo$he', 'ello$h'],
  'postings': ['hello', 'o$hell', 'lo$hel', 'llo$he', 'ello$h']},
 'bye': {'rotations': ['bye', 'e$by', 'ye$b'],
  'postings': ['bye', 'e$by', 'ye$b']},
 'brother': {'rotations': ['brother',
   'r$brothe',
   'er$broth',
   'her$brot',
   'ther$bro',
   'other$br',
   'rother$b'],
  'postings': ['brother',
   'r$brothe',
   'er$broth',
   'her$brot',
   'ther$bro',
   'other$br',
   'rother$b']},
 'hellow': {'rotations': ['hellow',
   'w$hello',
   'ow$hell',
   'low$hel',
   'llow$he',
   'ellow$h'],
  'postings': ['hellow',
   'w$hello',
   'ow$hell',
   'low$hel',
   'llow$he',
   'ellow$h']}}

In [105]:
db.keys()

dict_keys(['hello', 'bye', 'brother', 'hellow'])

In [127]:
def union(p1, p2):
    i = j = 0
    res = []
    
    while i < len(p1) and j < len(p2):
        if p1[i] == p2[j]:
            res.append(p1[i])
        elif p1[i] < p2[j]:
            res.append(p1[i])
            i += 1
        elif p1[i] > p2[j]:
            res.append(p2[j])
            j += 1
    if i < len(p1):
        res.append(p1[i:])
    else:
        res.append(p2[j:])
        
    return res

In [128]:
print(union([1,3,4,5], [1,2]))

KeyboardInterrupt: 

In [119]:
def match(term):
    res = []
    rotated, is_wild = rotate(term)
    if is_wild:
        for w in db.keys():
            print(w)
            if len(w) >= len(term)-1:
                for r in db[w]['rotations']:
                    if r[:min(len(term)-1, len(r))] == rotated:
                        res += db[w]['postings']
                        break
    else:
        for w in db.keys():
            if w == rotated:
                res += db[w]['postings']
                break
    return res

In [120]:
match("hello")

['hello', 'o$hell', 'lo$hel', 'llo$he', 'ello$h']

In [121]:
match("hello*")

hello
bye
brother
hellow


['hello',
 'o$hell',
 'lo$hel',
 'llo$he',
 'ello$h',
 'hellow',
 'w$hello',
 'ow$hell',
 'low$hel',
 'llow$he',
 'ellow$h']

In [23]:
print(rotate_perm("ans*r"))

r$ans


In [24]:
import numpy as np

In [40]:
def levenshtein_distance(word1, word2):
    l = max(len(word1), len(word2))
    m = np.zeros((l, l))
    for i in range(len(word1)):
        m[i, 0] = i
        
    for j in range(len(word2)):
        m[0, j] = j
    
    for i in range(1, len(word1)):
        for j in range(1, len(word2)):
            if word1[i] == word2[j]:
                m[i, j] = min(m[i-1, j] + 1, min(m[i, j-1] + 1, m[i-1, j-1]))
            else:
                m[i, j] = min(m[i-1, j] + 1, min(m[i, j-1] + 1, m[i-1, j-1] + 1))
    print(m)
    return m[len(word1)-1, len(word2)-1]

In [46]:
print(levenshtein_distance("rcaraa", "carsa"))

[[0. 1. 2. 3. 4. 0.]
 [1. 1. 2. 3. 4. 0.]
 [2. 1. 2. 3. 3. 0.]
 [3. 2. 1. 2. 3. 0.]
 [4. 3. 2. 2. 2. 0.]
 [5. 4. 3. 3. 2. 0.]]
2.0
